<a href="https://colab.research.google.com/github/wlg100/numseqcont_circuit_expms/blob/main/notebook_templates/minimal_circuit_template.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" align="left"/></a>&nbsp;or in a local notebook.

# Setup
(No need to change anything)

In [12]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-ygtusyr7
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-ygtusyr7
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit e9bbd6a95058ab0b5d6ba44223ee05564384f2ce
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━

In [13]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [14]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [15]:
import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

We turn automatic differentiation off, to save GPU memory, as this notebook focuses on model inference not model training.

In [16]:
torch.set_grad_enabled(False)

Plotting helper functions:

In [17]:
def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

# Load Model

Decide which model to use (eg. gpt2-small vs -medium)

In [18]:
model = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


# Import functions from repo

In [19]:
!git clone https://github.com/callummcdougall/ARENA_2.0.git

Cloning into 'ARENA_2.0'...
remote: Enumerating objects: 9006, done.
remote: Counting objects: 100% (9006/9006), done.
remote: Compressing objects: 100% (3496/3496), done.
remote: Total 9006 (delta 5468), reused 8861 (delta 5413), pack-reused 0
Receiving objects: 100% (9006/9006), 155.22 MiB | 15.37 MiB/s, done.
Resolving deltas: 100% (5468/5468), done.


In [20]:
cd ARENA_2.0/chapter1_transformers/exercises/part3_indirect_object_identification

/content/ARENA_2.0/chapter1_transformers/exercises/part3_indirect_object_identification


In [21]:
import ioi_circuit_extraction as ioi_circuit_extraction

# Generate dataset with multiple prompts

In [23]:
class Dataset:
    def __init__(self, prompts, tokenizer, S1_is_first=False):
        self.prompts = prompts
        self.tokenizer = tokenizer
        self.N = len(prompts)
        self.max_len = max(
            [
                len(self.tokenizer(prompt["text"]).input_ids)
                for prompt in self.prompts
            ]
        )
        # all_ids = [prompt["TEMPLATE_IDX"] for prompt in self.ioi_prompts]
        all_ids = [0 for prompt in self.prompts] # only 1 template
        all_ids_ar = np.array(all_ids)
        self.groups = []
        for id in list(set(all_ids)):
            self.groups.append(np.where(all_ids_ar == id)[0])

        texts = [ prompt["text"] for prompt in self.prompts ]
        self.toks = torch.Tensor(self.tokenizer(texts, padding=True).input_ids).type(
            torch.int
        )
        self.io_tokenIDs = [
            self.tokenizer.encode(" " + prompt["S5"])[0] for prompt in self.prompts
        ]
        self.s_tokenIDs = [
            self.tokenizer.encode(" " + prompt["S4"])[0] for prompt in self.prompts
        ]

        # word_idx: for every prompt, find the token index of each target token and "end"
        # word_idx is a tensor with an element for each prompt. The element is the targ token's ind at that prompt
        self.word_idx = {}
        for targ in [key for key in self.prompts[0].keys() if (key != 'text' and key != 'S5')]:
            targ_lst = []
            for prompt in self.prompts:
                input_text = prompt["text"]
                tokens = model.tokenizer.tokenize(input_text)
                if S1_is_first and targ == "S1":  # only use this if first token doesn't have space Ġ in front
                    target_token = prompt[targ]
                else:
                    target_token = "Ġ" + prompt[targ]
                target_index = tokens.index(target_token)
                targ_lst.append(target_index)
            self.word_idx[targ] = torch.tensor(targ_lst)

        targ_lst = []
        for prompt in self.prompts:
            input_text = prompt["text"]
            tokens = self.tokenizer.tokenize(input_text)
            end_token_index = len(tokens) - 1
            targ_lst.append(end_token_index)
        self.word_idx["end"] = torch.tensor(targ_lst)

    def __len__(self):
        return self.N

Repalce io_tokens with correct answer (next, which is '5') and s_tokens with incorrect (current, which repeats)

In [24]:
def generate_prompts_list(x ,y):
    prompts_list = []
    for i in range(x, y):
        prompt_dict = {
            'S1': str(i),
            'S2': str(i+1),
            'S3': str(i+2),
            'S4': str(i+3),
            'S5': str(i+4),
            'text': f"{i} {i+1} {i+2} {i+3}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

prompts_list = generate_prompts_list(1, 11)
dataset = Dataset(prompts_list, model.tokenizer, S1_is_first=True)

In [25]:
def generate_prompts_list_corr(x ,y):
    prompts_list = []
    for i in range(x, y):
        prompt_dict = {
            'S1': str(i),
            'S2': str(i+1),
            'S3': str(i+2),
            'S4': str(i+2),
            'S5': str(i+3),
            'text': f"{i} {i+1} {i+2} {i+2}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

prompts_list_2 = generate_prompts_list_corr(1, 11)
dataset_2 = Dataset(prompts_list_2, model.tokenizer, S1_is_first=True)

Logit diff is correct - incorr token. Here, correct is S5, and incorr is S4.

Because of this, it's possible to have logit diffs HIGHER than the "full circuit" because the correct token will still be at first place, but the logit scores assigned will just be bigger (perhaps incorrect is scored even lower in the non-full circuit with a higher logit diff score)?

# Ablate the model and compare with original

## Ablation Expm Functions

In [26]:
from torch import Tensor

def logits_to_ave_logit_diff_2(logits: Float[Tensor, "batch seq d_vocab"], dataset: Dataset, per_prompt=False):
    '''
    Returns logit difference between the correct and incorrect answer.

    If per_prompt=True, return the array of differences rather than the average.
    '''

    # Only the final logits are relevant for the answer
    # Get the logits corresponding to the indirect object / subject tokens respectively
    io_logits: Float[Tensor, "batch"] = logits[range(logits.size(0)), dataset.word_idx["end"], dataset.io_tokenIDs]
    s_logits: Float[Tensor, "batch"] = logits[range(logits.size(0)), dataset.word_idx["end"], dataset.s_tokenIDs]
    # Find logit difference
    answer_logit_diff = io_logits - s_logits
    return answer_logit_diff if per_prompt else answer_logit_diff.mean()

In [27]:
def mean_ablate_by_lst(lst, model, print_output=True):
    CIRCUIT = {
        "number mover": lst,
        "number mover 4": lst,
        "number mover 3": lst,
        "number mover 2": lst,
        "number mover 1": lst,
    }

    SEQ_POS_TO_KEEP = {
        "number mover": "end",
        "number mover 4": "S4",
        "number mover 3": "S3",
        "number mover 2": "S2",
        "number mover 1": "S1",
    }

    model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook

    ioi_logits_original, ioi_cache = model.run_with_cache(dataset.toks)

    model = ioi_circuit_extraction.add_mean_ablation_hook(model, means_dataset=dataset_2, circuit=CIRCUIT, seq_pos_to_keep=SEQ_POS_TO_KEEP)
    ioi_logits_minimal = model(dataset.toks)

    orig_score = logits_to_ave_logit_diff_2(ioi_logits_original, dataset)
    new_score = logits_to_ave_logit_diff_2(ioi_logits_minimal, dataset)
    if print_output:
        print(f"Average logit difference (IOI dataset, using entire model): {orig_score:.4f}")
        print(f"Average logit difference (IOI dataset, only using circuit): {new_score:.4f}")
    return new_score

## Ablate by combos in L0; test via Brute Force

Which combo of heads has the biggest contribution? Remember, looking at one head's combo in isolation isn't enough, as that head may have a huge contribution when paired when another head, and this "emergent" effect can't be guessed just by looking at adding them separately from one another.

We want to automate this search process by having the script check the score against a threshold. If we try brute force of 12 heads in L0 for all possible subset sizes, this is around 4000, which is feasible to compute (but does not scale up, esp when we want to find paths in later layers which depend on combinations ACROSS layers, not just in 1 layer). We can first try brute force and rank the top 10 combos.

In [28]:
from itertools import combinations

heads = range(12)
all_combinations = []

for r in range(1, len(heads) + 1):
    for combo in combinations(heads, r):
        all_combinations.append([(0, head) for head in combo])

print(all_combinations)

[[(0, 0)], [(0, 1)], [(0, 2)], [(0, 3)], [(0, 4)], [(0, 5)], [(0, 6)], [(0, 7)], [(0, 8)], [(0, 9)], [(0, 10)], [(0, 11)], [(0, 0), (0, 1)], [(0, 0), (0, 2)], [(0, 0), (0, 3)], [(0, 0), (0, 4)], [(0, 0), (0, 5)], [(0, 0), (0, 6)], [(0, 0), (0, 7)], [(0, 0), (0, 8)], [(0, 0), (0, 9)], [(0, 0), (0, 10)], [(0, 0), (0, 11)], [(0, 1), (0, 2)], [(0, 1), (0, 3)], [(0, 1), (0, 4)], [(0, 1), (0, 5)], [(0, 1), (0, 6)], [(0, 1), (0, 7)], [(0, 1), (0, 8)], [(0, 1), (0, 9)], [(0, 1), (0, 10)], [(0, 1), (0, 11)], [(0, 2), (0, 3)], [(0, 2), (0, 4)], [(0, 2), (0, 5)], [(0, 2), (0, 6)], [(0, 2), (0, 7)], [(0, 2), (0, 8)], [(0, 2), (0, 9)], [(0, 2), (0, 10)], [(0, 2), (0, 11)], [(0, 3), (0, 4)], [(0, 3), (0, 5)], [(0, 3), (0, 6)], [(0, 3), (0, 7)], [(0, 3), (0, 8)], [(0, 3), (0, 9)], [(0, 3), (0, 10)], [(0, 3), (0, 11)], [(0, 4), (0, 5)], [(0, 4), (0, 6)], [(0, 4), (0, 7)], [(0, 4), (0, 8)], [(0, 4), (0, 9)], [(0, 4), (0, 10)], [(0, 4), (0, 11)], [(0, 5), (0, 6)], [(0, 5), (0, 7)], [(0, 5), (0, 8)], [(0

In [29]:
len(all_combinations)

4095

In [30]:
base_lst = [(0, 1), (0, 10), (3, 0), (4, 4), (5, 5), (6, 1), (6, 6), (7, 10), (7, 11), (8, 8), (8, 11), (9, 1), (9, 5), (10, 7)]

In [31]:
all_scores = []

for combo in all_combinations:
    new_score = mean_ablate_by_lst(base_lst + combo, model, print_output=False)
    all_scores.append((combo, new_score))

KeyboardInterrupt: ignored

In [32]:
len(all_scores)

3470

Run time: 3470 found, so around 700 more to go. Est will take 2.5 hrs total.

In [33]:
# Sort scores and print top 10
sorted_scores = sorted(all_scores, key=lambda x: x[1], reverse=True)
for i, (combo, score) in enumerate(sorted_scores[:10]):
    print(f"{i + 1}. Combo: {combo} - Score: {score}")

1. Combo: [(0, 2), (0, 3), (0, 4), (0, 5), (0, 7), (0, 8), (0, 9)] - Score: 2.6552085876464844
2. Combo: [(0, 2), (0, 3), (0, 5), (0, 7), (0, 8), (0, 9), (0, 11)] - Score: 2.641260862350464
3. Combo: [(0, 2), (0, 3), (0, 5), (0, 7), (0, 8), (0, 9)] - Score: 2.626978635787964
4. Combo: [(0, 1), (0, 2), (0, 3), (0, 5), (0, 7), (0, 8), (0, 9)] - Score: 2.626978635787964
5. Combo: [(0, 2), (0, 3), (0, 5), (0, 7), (0, 8), (0, 9), (0, 10)] - Score: 2.626978635787964
6. Combo: [(0, 2), (0, 3), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9)] - Score: 2.624587297439575
7. Combo: [(0, 0), (0, 2), (0, 3), (0, 5), (0, 7), (0, 8), (0, 9)] - Score: 2.6186442375183105
8. Combo: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 5), (0, 7), (0, 8), (0, 9)] - Score: 2.6186442375183105
9. Combo: [(0, 2), (0, 4), (0, 5), (0, 7), (0, 8), (0, 9), (0, 11)] - Score: 2.6116158962249756
10. Combo: [(0, 2), (0, 4), (0, 5), (0, 7), (0, 8), (0, 9)] - Score: 2.597621202468872


In [34]:
import json

with open("scores.json", "w") as file:
    json.dump(all_scores, file, default=lambda x: str(x))  # Convert tuples to strings for JSON serialization

In [35]:
from google.colab import files
files.download("scores.json")  # or "scores.pkl" or "scores.json" depending on the file you saved

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

From the top, rank each by their length and find the top within each length sub-group.